## PROBLEM #1: Solving Reduced Hodgkin-Huxley Model Numerically 
In this problem you are asked to implement (solve) a reduced Hodgkin-Huxley neuron model (**Wilson neuron**) that is a further simplification of the Rinzel model.

\begin{equation}
\begin{split}
C\frac{dV}{dt} &= -g_{Na}\cdot(17.81+47.71V+32.63V^2)\cdot(V-E_{Na}) - g_R\cdot R\cdot(V-E_R) + I(t) \\
\tau_R\frac{dR}{dt} &= -R + 1.35V + 1.03 \\
\end{split}
\end{equation}
where 


    
|$g_{Na}$|$E_{Na}$|$g_R$|$E_R$|$\tau_R$|$C$|
|:----|:-----|:---|:---|:-----|:-----|
|<img style="width:100px">1|<img style="width:100px">0.55|<img style="width:100px">26|<img style="width:100px">-0.92|<img style="width:100px">1.9|<img style="width:100px">0.8|
    

Create an input current that is a periodic bandlimited function of the form
$$I(t)= \sum_{m=-M}^{M} a_m \cos \left(\frac{m\Omega t}{M}\right)$$
where $\Omega = 2 \pi \cdot 100$ Hz, $M=3$.

1. Implement the Wilson neuron model defined by the equation above. You are asked to implement [Euler's method](https://en.wikipedia.org/wiki/Euler_method) for solving differential equations numerically. Essentially, for differential equation $\frac{dx}{dt} = f(x) + I(t)$, we iteratively solve the value for $x(t+\Delta t)$ given previous value $x(t)$ and gradient $f(x(t)) + I(t)$ as $\frac{x(t+\Delta t) - x(t)}{\Delta t} = f(x(t)) + I(t)$.

2. Assuming that $a_m=constant, -M \le m \le M$, with initial conditions $V(0)=-0.7, R(0)=0.088$, plot the input and the output
of the Wilson neuron on the time interval [0, 200] ms. Empirically determine a range of values for the $a_m$'s that makes the neuron fire.
3. Plot the total number of spikes fired in the interval [0, 200] ms for a broad range of $a_m = constant$ values. For the same range of input current values, show the following
    1. spike count
    2. spike rate (spikes per second)
    3. spike amplitude (Voltage value at spike time)
4. Assume the input current is a [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function) $I(t) = c\cdot u(t)$, where $c$ is amplitude of the injected current in $pA$, find a range of value for $c$ where the Wilson neuron exhibits limit cycle (periodic spiking). Draw the limit cycle in the $V$ vs. $R$ plane for at least 3 different values of $c$, comment on the effect of the amplitude $c$ on the limit cycle based on your results.

## PROBLEM \#2 - Ion Channels of Hodgkin-Huxley Neuron
### Part 1
Assume that the input to a Hodgkin-Huxley neuron is an injected current of the form
$$I(t)= \sum^{10}_{k=0}a_k \frac{\sin \Omega (t-kT)}{\Omega (t-kT)}$$
where $a_k = constant, 0 \leq k \leq 10$, $\Omega = 2 \pi \cdot 60$ Hz, $T = \pi/\Omega$,
$t \in [0, 200]$ ms. 

1. Assuming that $a_k=constant, -0 \le k \le 10$, plot the input and the output
of the Hodgkin-Huxley neuron on the time interval [0, 200] ms with initial conditions $V(0) = -65, n(0) = m(0) = 0, h(0)=1$. Empirically determine a range of values for the $a_k$'s that makes the neuron fire robustly (at least more than once).

2. With the smallest $a_m$ that makes the neuron fire, plot the K, Na and capacitive currents, i.e., $I_K$, $I_{Na}$, and $C \frac{dV}{dt}$ as a function of time.


### Part 2

Use the membrane voltage $V$ of the Hodgkin-Huxley neuron obtained from Part 1 to define $V_K = V - \min(V)$. Then, using $V_K$ as input, simulate a transient Potassium memconductance (in isolation), defined by the following equations: 

\begin{equation}
\begin{split}
I_{K} &= \bar{g}_{K} \cdot a_{\infty}(V_K) \cdot b \cdot (V_K - E_{K}) \\
\tau_b \cdot \frac{db}{dt} &= b_{\infty}(V_K) - b \\
a_{\infty}(V_K) &= \frac{1}{1 + \exp{\left(-(V_K+27)/8.8 \right)}} \\
b_{\infty}(V_K) &= \frac{1}{1 + \exp{\left((V_K+68)/6.6 \right)}}
\end{split}
\end{equation}

Choose $\bar{g}_{K}=16$, $\tau_b=15$, $E_K=-90$ and simulate the above equations assuming that time is given in milliseconds.

Characterize the memconductance by plotting:

1. the input voltage $V_K$ and the resulting current $I_K$ as a function of time, and the internal state $b$ and the memcondactance $g_K$ as a function of time;
2. in a `2x2` array of subplots, plot the following:
    1. (top-left) memconductance $g_K$ versus voltage $V_K$,
    2. (top-right) memconductance $g_K$ versus flux $\phi_K$,
    3. (bottom-left) voltage $V_K$ versus current $I_K$,
    4. (bottom-right) charge $q_K$ vs. flux $\phi_K$
    
-----------------

### Setup

We import the numpy, fix its random seed, and import matplotlib for plot generation.

In [ ]:
# Import libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)  # fix random seed

We provided a convenience function for detecting spike, you can invoke this function as `spike_detect(V,35.0)`. Replace `35.0` by your desired threshold value.

In [ ]:
# Define spike detection function
def spike_detect(v, thresh):
    x = (v[2:-1] > v[1:-2]) * (v[2:-1] > v[3:]) * (v[2:-1] > thresh)
    x = np.append(False, x)
    x = np.append(x, False)
    return x

Define Function Generators for problem 1 and 2

In [ ]:
# TODO - define two functions for generating currents for question 1 and 2
# the functions should take the following forms

# problem_1_I_in(t,a,M,omega)
#   input: (t,a,M,omega), a is a vector of all am values
#   output: I

# problem_2_I_in(t,a,K,omega)
#   input: (t,a,K,omega), a is a vector of all ak values
#   output: I

# Problem 1 - Reduced Hodgkin Huxley Model

Specify global variables

In [ ]:
# TODO

t  = [] # time vector, in ms
M  = [] # order of signal
am = [] # coefficients of signal, set to all ones so we can scale them later
Omega = [] # bandwidth in rad/s = 2*pi*frequency

### Part 1 - Numerical Integration
Let us start by implementing the reduced Hodgkin Huxley Model. If you have trouble, please refer to `hodgkin_huxley.py` for refernece.

In [ ]:
def Wilson_neuron(t, I):
    """Simulate a Wilson neuron 
    # Parameters  
        t: duration, ms
        dt: temporal resolution, ms
        I: input current, pA 
    # Return Values
        V: Membrane potential over time, in V
        R: auxiliary variable R over time
    """
    # TODO - constant values
    g_Na = np.nan
    E_Na = np.nan
    g_R = np.nan
    E_R = np.nan
    tau_R = np.nan
    C = np.nan
    
    # TODO - compute dt
    dt = np.nan
    
    V = np.zeros_like(T) # voltage over time
    R = np.zeros_like(T) # R over time
    
    # TODO - Initialize V_0, R_0
    V[0] = np.nan
    R[0] = np.nan

    for i in range(1, len(t)): 
        # TODO - update rule 

    return V, R

### Part 2 - range of $a_m$
Then we plot the input current and the corresponding membrane voltage for two constants, with the constant $a_m=0.2$ and $a_m=0.4$.

In [ ]:
# TODO - run experiment with different scales 
scale = 1
# = problem_1_I_in(t, am*scale, M, Omega)

# we use the provided hogkinhuxley model to run the experiment
# and obtain spike states using the utility function defined above
V = Wilson_neuron(t,I)
S = spike_detect(V,0.2)

# plot the inputs and voltages (4 plots in total)
# (we recommend using subplots and PLEASE CLARIFY what each plot is with titles and axis-units)

Now, let us compute the number of spikes for a range of $a_m$ values.

In [ ]:
# TODO - count spikes only in in t=[0,200]ms
# we recommend using np.linspace or np.arrange to generate am's (you can look up documentations)

Let us now plot the spike counts and voltage traces for some current level. These count and voltage traces show that the empirical range that makes the neuron fire is approximately $a_m \in \{0.03, 2\}$. After passing the upper limit, we observe that the HH neuron has undefined behavior.

In [ ]:
# TODO - plot number of spikes for different values of am

### Part 3 - compare across $a_m$
For a range of $a_m$ values, show `count`,`rate`,`amplitude`

In [ ]:
# TODO - compare results across a_m constant values

### Part 4 -  Limit Cycle
In this part, you will need to change the input and plot the relation between V and R and determine whether there is a limit cycle by plotting V against R. 

In [ ]:
# TODO 

# please plot the LC with both the lower and higher bounds for c (that you determined)
# as well as at least one case in between

# Problem 2 - Ion Channels of Hodgkin-Huxley Model

Define the global parameters for the problem.

In [ ]:
# TODO
t = np.nan# in s, note that this is different from the problem 1
Omega = np.nan
ak = np.nan
T = pi/Omega
dt  = np.nan 

Generate input current for this section

In [ ]:
# TODO - Generate current for problem 2 using defined function
# e.g. I_ext = problem_2_I_in(t,a,K,omega)

### Part 1 - Ion Channel Currents
Let us start by performing simulation with different $a_k$ and determine the range that makes the neuron fire robustly (at least three spikes). 

In [ ]:
# TODO - simulate hodgkin-huxley model

With the smallest $a_k$, let the function return all intermediate values for potential, current and state.

In [ ]:
# TODO - simulate hodgkin-huxley model with smallest a_k

Plot the output potential (V) along with the requested currents.

In [ ]:
# TODO - generate plot

### Part 2 - Transient K channel

Simulate the transient potassium channel using voltage from the previous part as input.

In [ ]:
# TODO - simulate only transient K channel

In [ ]:
# TODO - Compute charge and flux 
# you can do numerical integration using cumsum, trapz or euler methods,
# example:
#    phi = np.cumsum(V_K)*dt
#    q = np.cumsum(I_K)*dt

In [ ]:
# TODO - generate plots 